In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture 

!pip install transformers
!pip install pytorch_pretrained_bert==0.4.0
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
! bash install_mecab-ko_on_colab190912.sh

## 데이터 불러오기




In [ ]:
%cd /content/drive/MyDrive/마이야르/6.Code 
import config
import pandas as pd

train = pd.read_csv(config.TRAIN_DIR)
test = pd.read_csv(config.TEST_DIR)
submission = pd.read_csv(config.SUBMISSION_DIR)

/content/drive/.shortcut-targets-by-id/1enR-TKNggkQ_bAdN7UF7isfRkeewUc_-/NH_dacon/예선/6.Code


## Library 불러오기

In [ ]:
import time
start = time.time()

import os
import sys
import random
import easydict 
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
from pytorch_pretrained_bert.modeling import BertForSequenceClassification as ETRI_BertForSequenceClassification
from pytorch_pretrained_bert.modeling import BertConfig, WEIGHTS_NAME, CONFIG_NAME

import re
import datetime
import warnings
from collections import Counter, OrderedDict

import numpy as np
from konlpy.tag import Mecab
from lightgbm import LGBMClassifier 
from sklearn.model_selection import train_test_split

import torch
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification

import module
import preprocess
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
warnings.filterwarnings(action='ignore')


## pos_Tagger, Tokenizer, pretrained_embedding, Model 불러오기

In [ ]:
'''
Tokenizer
'''
mecab = Mecab()

'''
BERT Multilingual Loading
'''
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
bert_ml_model = BertForSequenceClassification.from_pretrained(config.BERT_ML_PATH, num_labels=2)


'''
ETRI Model Loading
'''
vocab2id = preprocess.construct_vocab(config.VOCAB2ID_PATH)
args = easydict.EasyDict({"output_dir":config.ETRI_OUTPUT_DIR,
                          "max_seq_length":200,
                          "do_lower_case":False,
                          "eval_batch_size":32,
                          "no_cuda": False,
                          "local_rank":-1,
                          "seed":42})
test2 = test.copy()
train2  = train.copy()
label_list = ["0", "1"]
num_labels = num_labels_task = len(label_list)
output_config_file = os.path.join(args.output_dir, CONFIG_NAME)
output_model_file = os.path.join(args.output_dir, WEIGHTS_NAME)

bert_config = BertConfig(output_config_file)
bert_etri_model = ETRI_BertForSequenceClassification(bert_config, num_labels=num_labels)
bert_etri_model.load_state_dict(torch.load(output_model_file))

<All keys matched successfully>

## 형태소 분석 + 전처리

In [ ]:
'''
특수 문자 전처리
'''
train["title"] = train.apply(lambda x : preprocess.clean_text(x["title"]), axis=1)
train["content"] = train.apply(lambda x : preprocess.clean_text(x["content"]), axis=1)

test["title"] = test.apply(lambda x : preprocess.clean_text(x["title"]), axis=1)
test["content"] = test.apply(lambda x : preprocess.clean_text(x["content"]), axis=1)

'''
토크나이징
'''
train["title_tokenized"] = train["title"].apply(lambda x : preprocess.get_pos(x, mecab))
train["content_tokenized"] = train["content"].apply(lambda x : preprocess.get_pos(x, mecab))

test["title_tokenized"] = test["title"].apply(lambda x : preprocess.get_pos(x, mecab))
test["content_tokenized"] = test["content"].apply(lambda x : preprocess.get_pos(x, mecab))

'''
BAD tokens
'''
train["title_morphs"] = train['title_tokenized'].apply(lambda x : [token.split('/')[0] for token in x])
train["content_morphs"] = train['content_tokenized'].apply(lambda x : [token.split('/')[0] for token in x])

test["title_morphs"] = test['title_tokenized'].apply(lambda x : [token.split('/')[0] for token in x])
test["content_morphs"] = test['content_tokenized'].apply(lambda x : [token.split('/')[0] for token in x])

fake_tokens = train[train["info"] == 1]["content_morphs"]
fake_tokens = [a for b in fake_tokens for a in b]
fake_freq = Counter(fake_tokens)

real_tokens = train[train["info"] == 0]["content_morphs"]
real_tokens = [a for b in real_tokens for a in b]
real_freq = Counter(real_tokens)

bad_tokens = {}
for token, fake in fake_freq.items():
    if token in real_freq.keys():
        real = real_freq[token]
    else:
        real = 0
        
    if fake > real * 5 and fake > 20 and len(token) > 1:
        bad_tokens[token] = fake

'''
중복셋 전처리
'''
train_unq = train.drop_duplicates(subset='content')  # 학습셋 유니크
test_dup = test[test['content'].isin(train_unq['content'].values)] # 테스트셋과 중복 찾기
test_dup['idx'] = test_dup.index  # 원본 테스트셋 인덱스 보존
test_duplicated = pd.merge(test_dup, train_unq[['content', 'info']], on='content', how='left')  # 5만개 중복셋 고정 라벨 
test_unq = test[~test.index.isin(test_duplicated['idx'].values)]
test_idx = test_unq.index.values

train_unq2 = train2.drop_duplicates(subset='content')  
test_dup2 = test2[test2['content'].isin(train_unq2['content'].values)] 
test_dup2['idx'] = test_dup2.index  
test_duplicated2 = pd.merge(test_dup2, train_unq2[['content', 'info']], on='content', how='left') 
test_duplicated2 = test_duplicated2[test_duplicated2['info'] == 1]
test_unq2 = test2[~test2.index.isin(test_duplicated2['idx'].values)]
test_idx2 = test_unq2.index.values

## Feature Engineering

In [ ]:
# 하나의 타이틀에 대한 기사 개수 
total_train = pd.DataFrame(train.groupby(["n_id", "title"]).size()).reset_index().rename(columns={0 : 'total'})
train = pd.merge(train, total_train[["n_id", "total"]], on = 'n_id', how='left')    # 'total' feature 붙인 것 
total_test = pd.DataFrame(test.groupby(["n_id", "title"]).size()).reset_index().rename(columns={0 : 'total'})
test = pd.merge(test, total_test[["n_id", "total"]], on = 'n_id', how='left')       # 'total' feature 붙인 것 

# 제목에 특정 단어 포함
train["word1"] = train.apply(lambda x : x["title"].count('52주 신고가') + x["content"].count('52주 신고가'), axis=1)
train["word2"] = train["title"].apply(lambda x : 1 if x in ('급등 중') else 0)
train["word3"] = train["title"].apply(lambda x : 1 if re.search(r'\(\d{6}\)', x) else 0)
train["word4"] = train["title"].apply(lambda x : 1 if x in ('상승|하락') else 0)
train["word5"] = train.apply(lambda x : x["title"].count('급반등') + x["content"].count('급반등'), axis=1)
train["word6"] = train.apply(lambda x : x["title"].count('코로나') + x["content"].count('코로나'), axis=1)
train["word7"] = train.apply(lambda x : x["title"].count('유망') + x["content"].count('유망'), axis=1)
train["word8"] = train["title"].apply(lambda x : 1 if x in ('클릭|Click') else 0)
test["word1"] = test.apply(lambda x : x["title"].count('52주 신고가') + x["content"].count('52주 신고가'), axis=1)
test["word2"] = test["title"].apply(lambda x : 1 if x in ('급등 중') else 0)
test["word3"] = test["title"].apply(lambda x : 1 if re.search(r'\(\d{6}\)', x) else 0)
test["word4"] = test["title"].apply(lambda x : 1 if x in ('상승|하락') else 0)
test["word5"] = test.apply(lambda x : x["title"].count('급반등') + x["content"].count('급반등'), axis=1)
test["word6"] = test.apply(lambda x : x["title"].count('코로나') + x["content"].count('코로나'), axis=1)
test["word7"] = test.apply(lambda x : x["title"].count('유망') + x["content"].count('유망'), axis=1)
test["word8"] = test["title"].apply(lambda x : 1 if x in ('클릭|Click') else 0)

# POS tagger morphs
train["title_tkn_wo_pos"] = train['title_tokenized'].apply(lambda x : [token.split('/')[0] for token in x])
test["title_tkn_wo_pos"] = test['title_tokenized'].apply(lambda x : [token.split('/')[0] for token in x])
train["content_tkn_wo_pos"] = train['content_tokenized'].apply(lambda x : [token.split('/')[0] for token in x])
test["content_tkn_wo_pos"] = test['content_tokenized'].apply(lambda x : [token.split('/')[0] for token in x])

# BAD tokens: 가짜뉴스에 많은 단어 
fake_tokens = train[train["info"] == 1]["content_tkn_wo_pos"]
f_tokens = [a for b in fake_tokens for a in b]
f_tokens = Counter(f_tokens)
fake_freq = pd.Series(f_tokens).sort_values(ascending=False)
real_tokens = train[train["info"] == 0]["content_tkn_wo_pos"]
r_tokens = [a for b in real_tokens for a in b]
r_tokens = Counter(r_tokens)
real_freq = pd.Series(r_tokens).sort_values(ascending=False)

bad_tokens = {}
for token, fake in zip(fake_freq.index, fake_freq):
    if token in real_freq.index:
        real = real_freq.loc[token]
    else:
        real = 0
        
    if fake > real * 5 and fake > 20 and len(token) > 1:
        bad_tokens[token] = fake
        
train["title_bad"] = train["title_tkn_wo_pos"].apply(lambda x : preprocess.count_badtokens(x, bad_tokens))
test["title_bad"] = train["title_tkn_wo_pos"].apply(lambda x : preprocess.count_badtokens(x, bad_tokens))
train["content_bad"] = train["content_tkn_wo_pos"].apply(lambda x : preprocess.count_badtokens(x, bad_tokens))
test["content_bad"] = train["content_tkn_wo_pos"].apply(lambda x : preprocess.count_badtokens(x, bad_tokens))

# Content 길이
train["content_length"] = train.apply(lambda x : len(x["content"]), axis=1)
train["content_words"] = train.apply(lambda x : len(x["content"].split()), axis=1)
train["content_tokens"] = train.apply(lambda x : len(x["content_tokenized"]), axis=1)
test["content_length"] = test.apply(lambda x : len(x["content"]), axis=1)
test["content_words"] = test.apply(lambda x : len(x["content"].split()), axis=1)
test["content_tokens"] = test.apply(lambda x : len(x["content_tokenized"]), axis=1)

# 괄호
train["bracket1"] = train["title"].apply(lambda x : 1 if x in ('\(') else 0)
train["bracket2"] = train["title"].apply(lambda x : 1 if x in ('\(|\<|\【') else 0)
test["bracket1"] = test["title"].apply(lambda x : 1 if x in ('\(') else 0)
test["bracket2"] = test["title"].apply(lambda x : 1 if x in ('\(|\<|\【') else 0)

# 기사개수
train["total1"] = train["total"].apply(lambda x : 1 if x==1 else 0)
test["total1"] = test["total"].apply(lambda x : 1 if x==1 else 0)
train["total5"] = train["total"].apply(lambda x : 1 if 1<x<=5 else 0)
test["total5"] = test["total"].apply(lambda x : 1 if 1<x<=5 else 0)

# Ord
train["ord18"] = train["ord"].apply(lambda x : 1 if x <= 18 else 0)
test["ord18"] = test["ord"].apply(lambda x : 1 if x <= 18 else 0)

# 날짜 관련 통계량
count = pd.DataFrame(train.groupby(["n_id", "title", "info"]).size()).reset_index().rename(columns={0 : 'count'})     
fake = count[count["info"] == 1].rename(columns={'count' : 'fake'})
real = count[count["info"] == 0].rename(columns={'count' : 'real'}) 
train = pd.merge(train, fake[["n_id", "fake"]], on = 'n_id', how='left')    # fake 
train = pd.merge(train, real[["n_id", "real"]], on = 'n_id', how='left')    # real 

def ratio(a, b) : 
    return a / b
train["fake_ratio"] = train.apply(lambda x : ratio(x["fake"], x["total"]), axis=1)  # 가짜 / 전체 
train["real_ratio"] = train.apply(lambda x : ratio(x["real"], x["total"]), axis=1)  # 진짜 / 전체 
temp = train.groupby('date').agg({
    'fake_ratio' : [('fake_ratio_max', np.max),
                    ('fake_ratio_min', np.min),
                    ('fake_ratio_mean', np.mean),
                    ('fake_ratio_median', np.median)
                    ], 
    'real_ratio' : [('real_ratio_max', np.max),
                    ('real_ratio_min', np.min),
                    ('real_ratio_mean', np.mean),
                    ('real_ratio_median', np.median)
                    ],     
}).reset_index()
train = train.merge(temp, on = 'date', how = 'left')
test = test.merge(temp, on = 'date', how = 'left')

# 자카드 유사도
train["jaccard_similarity"] = train.apply(lambda x : preprocess.jaccard_sim(x["title_tkn_wo_pos"], x["content_tkn_wo_pos"]), axis = 1)
test["jaccard_similarity"] = test.apply(lambda x : preprocess.jaccard_sim(x["title_tkn_wo_pos"], x["content_tkn_wo_pos"]), axis = 1)

train_feature = train.drop(['date', 'title', 'content', 'real', 'fake', 'fake_ratio', 'real_ratio', 'title_tokenized', 'content_tokenized', \
                            'title_morphs', 'content_morphs', 'title_processed', 'content_processed', 'title_tkn_wo_pos', 'content_tkn_wo_pos'], axis=1, errors='ignore')
test_feature = test.drop(['date', 'title', 'content', 'real', 'fake', 'fake_ratio', 'real_ratio', 'title_tokenized', 'content_tokenized', \
                            'title_morphs', 'content_morphs', 'title_processed', 'content_processed', 'title_tkn_wo_pos', 'content_tkn_wo_pos'], axis=1, errors='ignore')

## 예측

### LGBM

In [ ]:
X = train_feature.drop(["n_id", "info"], axis=1)
y = train_feature["info"]
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 2020, test_size=0.2)

lgbm = LGBMClassifier(n_estimators=200)
lgbm.fit(X_train, y_train)
lgbm_preds = lgbm.predict(test_feature.drop(["n_id", "id"], axis=1))

### Bert Classifier - multilingual

In [ ]:
'''
Dataloader 
'''
MAX_LEN = 128
torch.cuda.empty_cache()

content_processed = ['[CLS] ' + title + ' [SEP] ' + content + ' [SEP] ' for title, content in zip(test_unq['title'].values, test_unq['content'].values)]

tokens = [tokenizer.tokenize(sent) for sent in content_processed]
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokens]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post", value=1)
attention_masks = [[float(i != 1) for i in ids] for ids in input_ids]

inputs = torch.tensor(input_ids)
masks = torch.tensor(attention_masks)

dataset = TensorDataset(inputs, masks)
dataloader = DataLoader(dataset, batch_size = 32)

'''
Inference
'''
bert_ml_model = bert_ml_model.to(device)
bert_ml_model.eval()

ml_preds = [] 
for b_input_ids, b_input_mask in dataloader:
    b_input_ids = b_input_ids.to(device)
    b_input_mask = b_input_mask.to(device)

    with torch.no_grad():     
        outputs = bert_ml_model(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask)

    logits = outputs[0].detach().cpu().numpy()
    pred = np.argmax(logits, axis=1)  # list  
    ml_preds.extend(pred)
    
ml_pred = {i: x for i, x in zip(test_idx, ml_preds)}
ml_dup = {i: x for i, x in zip(test_duplicated['idx'].values, test_duplicated['info'].values)}
ml_pred.update(ml_dup)

bert_ml_preds = list(OrderedDict(sorted(ml_pred.items())).values())

### Bert Classifier - ETRI

In [ ]:
'''
ETRI Model Setting
'''
torch.cuda.empty_cache()
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)

processors = module.ColaProcessor(title_column='title_tokenized2', content_column='content_tokenized2')
    
if n_gpu > 0:
    torch.cuda.manual_seed_all(args.seed)

'''
ETRI Model Inference
'''
test2 = pd.read_csv(config.TEST_DIR)
train2 = pd.read_csv(config.TRAIN_DIR)
train_unq2 = train2[train2['info'] == 1]
train_unq2 = train_unq2.drop_duplicates(subset='content')  
test_dup2 = test2[test2['content'].isin(train_unq2['content'].values)] 
test_dup2['idx'] = test_dup2.index  
test_duplicated2 = pd.merge(test_dup2, train_unq2[['content', 'info']], on='content', how='left') 
test_unq2 = test2[~test2.index.isin(test_duplicated2['idx'].values)]
test_idx2 = test_unq2.index.values

test_unq2["title_tokenized2"] = test_unq2["title"].apply(lambda x : preprocess.get_pos(x, mecab))
test_unq2["content_tokenized2"] = test_unq2["content"].apply(lambda x : preprocess.get_pos(x, mecab))

test_examples = processors.get_test_examples(test_unq2)
test_features = module.convert_examples_to_features(
    test_examples, label_list, args.max_seq_length, preprocess.get_pos, vocab2id)

input_ids = torch.LongTensor([f.input_ids for f in test_features])
input_masks = torch.LongTensor([f.input_mask for f in test_features])
segment_ids = torch.LongTensor([f.segment_ids for f in test_features])

eval_data = TensorDataset(input_ids, input_masks, segment_ids)
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size)

bert_etri_model = bert_etri_model.to(device)
bert_etri_model.eval()

etri_preds = []
for input_id, input_mask, segment_id in eval_dataloader:
    input_id = input_id.to(device)
    input_mask = input_mask.to(device)
    segment_id = segment_id.to(device)

    with torch.no_grad():
        outputs = bert_etri_model(input_id, segment_id, input_mask)

    logits = outputs.detach().cpu().numpy()
    pred = np.argmax(logits, axis=1)
    etri_preds.extend(pred)
    
etri_pred = {i: x for i, x in zip(test_idx2, etri_preds)}
etri_dup = {i: x for i, x in zip(test_duplicated2['idx'].values, test_duplicated2['info'].values)}
etri_pred.update(etri_dup)

bert_etri_preds = list(OrderedDict(sorted(etri_pred.items())).values())

### Ensemble

In [ ]:
ensemble = [] 

for a, b, c in zip(bert_etri_preds, bert_ml_preds, lgbm_preds): 
    if a == b == c : 
        ensemble.append(a)
    elif a != b : 
        ensemble.append(c)
    elif (a == b and b != c) : 
        ensemble.append(a)

submission['info'] = ensemble

## 제출

In [ ]:
print(time.time() - start)

submission.to_csv('/content/submission.csv')